https://colab.research.google.com/drive/1agkzI7fbYr4iJdFiIOIa5B3hk3_Yepot?usp=sharing

# Setup

In [10]:
import warnings

# Ignore UserWarnings
warnings.filterwarnings('ignore', category=UserWarning)

# Ignore TensorFlow warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

## Download Dataset

In [1]:
!pip3 install --upgrade gdown
!gdown https://drive.google.com/uc?id=1HupS43P60W1vzBxEvNMmKJubum9QqY_r
!unzip -q /content/data_v2.zip

  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
Downloading...
From (uriginal): https://drive.google.com/uc?id=1HupS43P60W1vzBxEvNMmKJubum9QqY_r
From (redirected): https://drive.google.com/uc?id=1HupS43P60W1vzBxEvNMmKJubum9QqY_r&confirm=t&uuid=84fad731-ba15-408b-b87b-c3dbc9cdab4a
To: /content/data_v2.zip
100% 370M/370M [00:04<00:00, 78.1MB/s]
unzip:  cannot find or open content/data_v2.zip, content/data_v2.zip.zip or content/data_v2.zip.ZIP.


In [3]:
import gdown
gdown.download_folder("https://drive.google.com/drive/folders/1-CKKgpis2PShA3wQL1F9Rkq5i-Yv6hCm?usp=drive_link", quiet=True)
gdown.download_folder("https://drive.google.com/drive/folders/1QiTPba9c916OZLXHU9c3F_VN_TQ-PEYm?usp=sharing", quiet=True)
gdown.download_folder("https://drive.google.com/drive/folders/1Eq9vKx2fNiG1zSA6CHRS4cSNJrvu5soI?usp=sharing",quiet=True)
gdown.download_folder("https://drive.google.com/drive/u/0/folders/1Ls1pBGEqN0cB3Hz5RNsSAlDQuF707CUK",quiet=True)

['/content/resnet50v2 TL/fingerprint.pb',
 '/content/resnet50v2 TL/keras_metadata.pb',
 '/content/resnet50v2 TL/saved_model.pb',
 '/content/resnet50v2 TL/variables/variables.data-00000-of-00001',
 '/content/resnet50v2 TL/variables/variables.index']

In [4]:

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import numpy as np
import time
import os
import math
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split

## INIT VALIDATOR

In [5]:
# Path to the data directory
data_dir = Path("./data_v2/")#("./captcha_images_v2/")

# Get list of all the images
images = sorted(list(map(str, list(data_dir.glob("*.jpg")))))
labels = [img.split(os.path.sep)[-1].split(".jpg")[0] for img in images]
characters = set(char for label in labels for char in label)#Set trong Python là tập hợp các phần tử duy nhất, không có thứ tự
characters = sorted(list(characters))

print("Number of images found: ", len(images))
print("Number of labels found: ", len(labels))
print("Number of unique characters: ", len(characters))
print("Characters present: ", characters)

# Batch size for training and validation
batch_size = 512

# Desired image dimensions
img_width = 200
img_height = 50

# Maximum length of any captcha in the dataset
max_length = max([len(label) for label in labels])
X_train,X_val_test,y_train,y_val_test=train_test_split(images,labels,test_size=0.2,random_state=2023)
X_val,X_test,y_val,y_test=train_test_split(X_val_test,y_val_test,test_size=0.5,random_state=2023)

x_train=np.array(X_train)
x_val=np.array(X_val)
x_test=np.array(X_test)

y_train=np.array(y_train)
y_val=np.array(y_val)
y_test=np.array(y_test)
# Mapping characters to integers
char_to_num = layers.StringLookup(
    vocabulary=list(characters), mask_token=None
)

# Mapping integers back to original characters
num_to_char = layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)



def encode_single_sample(img_path, label):
    # 1. Read image
    img = tf.io.read_file(img_path)
    #print(img)
    # 2. Decode and convert to grayscale
    img = tf.io.decode_png(img, channels=3)#channels=1 để chuyển sang thang độ xám, để 3 nếu muốn nahr màu


    #print(img)
    # 3. Convert to float32 in [0, 1] range
    img = tf.image.convert_image_dtype(img, tf.float32)
    #print(img)
    # 4. Resize to the desired size
    img = tf.image.resize(img, [img_height, img_width])
    #print(img.shape)
    # 5. Transpose the image because we want the time
    # dimension to correspond to the width of the image.
    img = tf.transpose(img, perm=[1, 0, 2])# chuyển vị matrix hàng thành cột và cột thành hàng
    #print(img.shape)
    # 6. Map the characters in label to numbers
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
    # 7. Return a dict as our model is expecting two inputs
    return {"image": img, "label": label}
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = (
    train_dataset.map(
        encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE
    )
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

validation_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
validation_dataset = (
    validation_dataset.map(
        encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE
    )
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = (
    test_dataset.map(
        encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE
    )
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][
        :, :max_length
    ]
    # Iterate over the results and get back the text
    output_text = []
    for res in results:
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("utf-8")
        output_text.append(res)
    return output_text

Number of images found:  113062
Number of labels found:  113062
Number of unique characters:  60
Characters present:  ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [19]:
def getACC(model,dataset,_y_true):
  pred = model.predict(dataset,verbose = 0)
  pred_texts = decode_batch_predictions(pred)
  y_true = _y_true
  correct_char = 0
  total_char = 0
  correct = 0
  #xét từng cặp true,pred
  # nếu true=pred thì số nhãn đoán đúng +1
  # với mỗi kí tự trong true ss với kí tự trong pred ở vị trí tương ứng, nếu bằng số kí tự đúng +1
  for i in range(len(y_true)):
      pr = pred_texts[i]
      tr = y_true[i]
      total_char += len(tr)
      for j in range(min(len(tr), len(pr))):
          if tr[j] == pr[j]:
              correct_char += 1

      if pr == tr :
          correct += 1

  print('Correct characters predicted : %.2f%%' %(correct_char*100/total_char))#nếu muốn tính % sai hãy dùng Edit distance
  print('Correct words predicted      : %.2f%%' %(correct*100/len(y_true)))

##LOAD TRAINED MODEL


In [16]:
def initial(model):
  image = cv2.imread('/content/model_ALexNet/Capture.PNG')
  tensor=image[:, :, :3]/255
  tensor= tf.image.transpose(tensor)
  tensor = tf.image.resize(tensor, [200, 50])
  image =tf.image.resize(tensor, [200, 50])
  _image= tf.reshape(image, (1, 200, 50, 3))
  model.predict(_image, verbose=0)
def load_model_CNN():
  model=tf.keras.saving.load_model("/content/CNN/ocr_model")
  initial(model)
  return model

def load_model_Res():
  model= tf.keras.saving.load_model("/content/resnet50v2.keras")
  initial(model)
  return model
def load_model_Res_TL():
  model= tf.keras.saving.load_model("/content/resnet50v2 TL")
  initial(model)
  return model
def load_model_Alex():
  model= tf.keras.saving.load_model("/content/model_ALexNet")
  initial(model)
  return model
CNN_model_pred=load_model_CNN()
RES_model_pred=load_model_Res()
RES_TL_model_pred=load_model_Res_TL()
ALEX_mode_pred=load_model_Alex()

# get Result

## With Val Dataset

In [ ]:
#@title get Accuracy
#CNN
print("CNN")
getACC(CNN_model_pred,validation_dataset,y_val)

#Alex
print("Alex")
getACC(ALEX_mode_pred,validation_dataset,y_val)

#ResNET Feature Extracter
print("RES FE")
getACC(RES_model_pred,validation_dataset,y_val)

#ResNET TransferLearning
print("RES TL")
getACC(RES_TL_model_pred,validation_dataset,y_val)


CNN


## With Test Dataset

In [20]:
#@title get Accuracy
#CNN
print("CNN")
getACC(CNN_model_pred,test_dataset,y_test)

#Alex
print("Alex")
getACC(ALEX_mode_pred,test_dataset,y_test)

#ResNET Feature Extracter
print("RES FE")
getACC(RES_model_pred,test_dataset,y_test)

#ResNET TransferLearning
print("RES TL")
getACC(RES_TL_model_pred,test_dataset,y_test)


CNN
Correct characters predicted : 89.36%
Correct words predicted      : 69.41%
Alex
Correct characters predicted : 84.92%
Correct words predicted      : 58.52%
RES FE
Correct characters predicted : 70.31%
Correct words predicted      : 23.83%
RES TL
Correct characters predicted : 89.04%
Correct words predicted      : 69.57%
